In [3]:
%pip install pandas pandasql

Note: you may need to restart the kernel to use updated packages.


In [29]:
import pandas as pd

import pandasql as ps

def  pysqldf(q):
    return ps.sqldf(q, globals())


pd.set_option('display.float_format', '{:.2f}'.format)


In [5]:
# extraccion de los datos

clientes = pd.read_csv('cliente.csv', sep=';')
facturas = pd.read_csv('factura.csv', sep=';')
despachos = pd.read_csv('despacho.csv', sep=';')
recogidas = pd.read_csv('recogida.csv', sep=';')
lotes = pd.read_csv('lote.csv', sep=';')
fincas = pd.read_csv('finca.csv', sep=';')
cultivos = pd.read_csv('cultivo.csv', sep=';')
precios = pd.read_csv('precio.csv', sep=';')
usuarios = pd.read_csv('usuario.csv', sep=';')

In [33]:
# 5. calcular el ingreso total por finca. 
# El ingreso total se calcula como la cantidad despachada por el 
# precio que tenía ese cultivo en el mes que se hizo la recogida.


# calcular el precio por mes por cultivo
precios_por_mes = pysqldf("""
        select 
        c.nombre as cultivo,
        strftime('%Y-%m', p.fecha) as mes,
        avg(p.valor) as precio_promedio
        from precios p
            join cultivos c
                on p.id_cultivo = c.id
        group by c.nombre, strftime('%Y-%m', p.fecha)
        """)


# calcular el total recogido por cultivo, finca y mes
total_recogido = pysqldf("""
        select 
        f.nombre as finca,
        c.nombre as cultivo,
        strftime('%Y-%m', r.fecha) as mes,
        sum(r.cantidad) as cantidad_recogida
        from recogidas r
            join lotes l
                on r.id_lote = l.id
            join fincas f
                on l.id_finca = f.id
            join cultivos c
                on l.id_cultivo = c.id
        group by f.nombre, c.nombre, strftime('%Y-%m', r.fecha)
        """)

# calcular el ingreso por mes por cultivo por finca
ingreso_mes_cultivo_finca = pysqldf("""
        select 
        tr.finca,
        tr.cultivo,
        tr.mes,
        tr.cantidad_recogida,
        ppm.precio_promedio,
        ppm.precio_promedio * tr.cantidad_recogida as ingreso
        from precios_por_mes ppm
            join total_recogido tr
                on ppm.cultivo = tr.cultivo and ppm.mes = tr.mes
        """)

ingreso_por_finca = pysqldf("""
    select
    finca,
    sum(ingreso) as total_ingreso    
    from ingreso_mes_cultivo_finca
    group by finca
    """)


ingreso_por_cultivo = pysqldf("""
    select
    cultivo,
    sum(ingreso) as total_ingreso    
    from ingreso_mes_cultivo_finca
    group by cultivo
    """)


ingreso_por_mes = pysqldf("""
    select
    mes,
    sum(ingreso) as total_ingreso    
    from ingreso_mes_cultivo_finca
    group by mes
    """)



In [35]:
ingreso_por_finca

,finca,total_ingreso
0,El Encanto,36795309.93
1,El Resguardo,8452692.17
2,La Esperanza,39128260.70
3,La Ilusión,69379159.92
4,La Prevalencia,107289206.26


In [32]:
ingreso_por_cultivo

,cultivo,total_ingreso
0,algodón,67378455.08
1,arroz,9946202.74
2,café,97647402.69
3,caña,3839229.31
4,maíz,13469892.74
5,palma africana,38068264.86
6,papa,3842338.16
7,sorgo,5392705.47
8,soya,15227854.97
9,trigo,6232282.95


In [34]:
ingreso_por_mes

,mes,total_ingreso
0,2021-01,8139591.71
1,2021-02,7257997.63
2,2021-03,7839360.55
3,2021-04,7493624.19
4,2021-05,8060379.03
5,2021-06,7798454.26
6,2021-07,8033219.76
7,2021-08,7886411.97
8,2021-09,8027475.27
9,2021-10,7937907.29


In [42]:
def execute_sql(file_name):

    path = f'./queries/{file_name}.sql'

    with open(path, 'r') as file:
        query = file.read()
    
    return pysqldf(query)

In [45]:
# 5. calcular el ingreso total por finca. 
# El ingreso total se calcula como la cantidad despachada por el 
# precio que tenía ese cultivo en el mes que se hizo la recogida.


# calcular el precio por mes por cultivo
precios_por_mes = execute_sql('precios_por_mes')

# calcular el total recogido por cultivo, finca y mes
total_recogido = execute_sql('total_recogido')

# calcular el ingreso por mes por cultivo por finca
ingreso_mes_cultivo_finca = execute_sql('ingreso_mes_cultivo_finca')

# calcular el ingreso por mes por finca
ingreso_por_finca = execute_sql('ingreso_por_finca')

# calcular el ingreso por mes por cultivo
ingreso_por_cultivo = execute_sql('ingreso_por_cultivo')

# calcular el ingreso por mes por mes
ingreso_por_mes =  execute_sql('ingreso_por_mes')